# Support Vector Regression (SVR)

## Importing the libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

## Importing the dataset

In [2]:
dataset = pd.read_csv('./data/1000_Companies.csv')
dataset.head()

,R&D Spend,Administration,Marketing Spend,State,Profit
0,165349.20,136897.80,471784.10,New York,192261.83
1,162597.70,151377.59,443898.53,California,191792.06
2,153441.51,101145.55,407934.54,Florida,191050.39
3,144372.41,118671.85,383199.62,New York,182901.99
4,142107.34,91391.77,366168.42,Florida,166187.94


In [3]:
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, -1].values

In [4]:
X[:5, :]

array([[165349.2, 136897.8, 471784.1, 'New York'],
       [162597.7, 151377.59, 443898.53, 'California'],
       [153441.51, 101145.55, 407934.54, 'Florida'],
       [144372.41, 118671.85, 383199.62, 'New York'],
       [142107.34, 91391.77, 366168.42, 'Florida']], dtype=object)

In [5]:
y[:5]

array([192261.83, 191792.06, 191050.39, 182901.99, 166187.94])

## Preprocess data

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1/5, random_state = 100)

In [7]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

X_preprocessor = ColumnTransformer(
    remainder='passthrough',        # Keep data that isn't processed by the transformer
    transformers=[
        ('numerical', StandardScaler(), [0, 1, 2]),
        ('categorical', OneHotEncoder(drop='first'), [3])
    ])

y_preprocessor = StandardScaler()

In [8]:
X_train_encoded = np.array(X_preprocessor.fit_transform(X_train))
X_test_encoded = np.array(X_preprocessor.transform(X_test))

display(X_train_encoded[:5, :])

array([[-1.48052312, -0.81802057, -1.45802246,  1.        ,  0.        ],
       [ 0.66635014,  0.31324176,  0.65534834,  1.        ,  0.        ],
       [ 0.01754759, -0.02863491,  0.01667051,  0.        ,  0.        ],
       [ 1.60033382,  0.80539031,  1.57475699,  0.        ,  0.        ],
       [-0.68956757, -0.40123853, -0.67940995,  1.        ,  0.        ]])

In [9]:
y_train = y_train.reshape(-1, 1)
y_test = y_test.reshape(-1, 1)

y_train = y_preprocessor.fit_transform(y_train)
y_test = y_preprocessor.transform(y_test)

display(y_train[:5])

array([[-1.37273435],
       [ 0.58663659],
       [-0.00550123],
       [ 1.43904857],
       [-0.65085879]])

## Training the model

In [10]:
from sklearn.svm import SVR
regressor = SVR(kernel = 'rbf')
regressor.fit(X_train_encoded, y_train)

/home/minh/.local/lib/python3.8/site-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


SVR()

## Evaluate model

In [11]:
accuracy = regressor.score(X_test_encoded, y_test)
print(f'Accuracy = {round(accuracy, 4) * 100}%')

Accuracy = 95.11%


## Predict

In [12]:
data = [
    [200000.0, 200000.0, 200000.0, 'New York'],
    [600000.0, 0.0, 0.0, 'California'],
    [0.0, 600000.0, 0.0, 'Florida'],
    [0.0, 0.0, 600000.0, 'New York']
    ]

In [13]:
data_scaled = X_preprocessor.transform(data)
y_pred_scaled = regressor.predict(data_scaled)
y_pred = y_preprocessor.inverse_transform(y_pred_scaled)

display(y_pred)

array([136679.17069133, 136078.69092858, 136078.69092858, 136077.68452454])